In [18]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.datasets import fetch_openml

In [19]:
data = fetch_openml("titanic", version=1, as_frame=True)
df = data.frame

In [20]:
from sklearn.model_selection import train_test_split

In [21]:
df.drop(['name', 'ticket'], axis=1, inplace=True)
df['boat'] = df['boat'].apply(lambda x: '0' if x is None else x)
def drop_nan(df):
    threshold = round(len(df) / 4)
    df.dropna(axis=1, inplace=True, thresh=threshold)
    df.dropna(axis=0, inplace=True)
drop_nan(df)

In [22]:
val_dataframe = df.sample(frac=0.2, random_state=1337)
train_dataframe = df.drop(val_dataframe.index)
df.head()

,pclass,survived,sex,age,sibsp,parch,fare,embarked,boat,home.dest
0,1.0,1,female,29.0000,0.0,0.0,211.3375,S,2,"St Louis, MO"
1,1.0,1,male,0.9167,1.0,2.0,151.5500,S,11,"Montreal, PQ / Chesterville, ON"
2,1.0,0,female,2.0000,1.0,2.0,151.5500,S,0,"Montreal, PQ / Chesterville, ON"
3,1.0,0,male,30.0000,1.0,2.0,151.5500,S,0,"Montreal, PQ / Chesterville, ON"
4,1.0,0,female,25.0000,1.0,2.0,151.5500,S,0,"Montreal, PQ / Chesterville, ON"


In [48]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler

In [33]:
y = df.pop('survived')

In [65]:
ct = ColumnTransformer(
    [('sex_', OrdinalEncoder(), ['sex']),
    ('embarked_', OrdinalEncoder(), ['embarked']),
    ('home_dest', OrdinalEncoder(), ['home.dest']),
    ('boat', OrdinalEncoder(), ['boat'])],
    remainder=MinMaxScaler()
)

# from sklearn.pipeline import Pipeline

# estimator = [('column_transf', ct), ('minmax', MinMaxScaler())]
# pipe = Pipeline(estimator)

X = ct.fit_transform(df)

In [37]:
y = y.to_numpy()

In [66]:
X[0, :]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [67]:
def dataframe_to_dataset(dataframe):
    dataframe = dataframe.copy()
    labels = dataframe.pop("survived")
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    ds = ds.shuffle(buffer_size=len(dataframe))
    return ds


train_ds = tf.data.Dataset.from_tensor_slices((np.asarray(X_train).astype('float32'), y_train))
test_ds = tf.data.Dataset.from_tensor_slices((np.asarray(X_test).astype('float32'), y_test))

In [68]:
train_ds = train_ds.batch(32)
val_ds = test_ds.batch(32)

In [25]:
from tensorflow.keras.layers.experimental.preprocessing import IntegerLookup
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras.layers.experimental.preprocessing import StringLookup


def encode_numerical_feature(feature, name, dataset):
    # Create a Normalization layer for our feature
    normalizer = Normalization()

    # Prepare a Dataset that only yields our feature
    feature_ds = dataset.map(lambda x, y: x[name])
    feature_ds = feature_ds.map(lambda x: tf.expand_dims(x, -1))

    # Learn the statistics of the data
    normalizer.adapt(feature_ds)

    # Normalize the input feature
    encoded_feature = normalizer(feature)
    return encoded_feature


def encode_categorical_feature(feature, name, dataset, is_string):
    lookup_class = StringLookup if is_string else IntegerLookup
    # Create a lookup layer which will turn strings into integer indices
    lookup = lookup_class(output_mode="binary")

    # Prepare a Dataset that only yields our feature
    feature_ds = dataset.map(lambda x, y: x[name])
    feature_ds = feature_ds.map(lambda x: tf.expand_dims(x, -1))

    # Learn the set of possible string values and assign them a fixed integer index
    lookup.adapt(feature_ds)

    # Turn the string input into integer indices
    encoded_feature = lookup(feature)
    return encoded_feature

In [26]:
sex = keras.Input(shape=(1,), name="sex", dtype="int64")
age = keras.Input(shape=(1,), name="age", dtype="int64")
sex = keras.Input(shape=(1,), name="sex", dtype="int64")
sex = keras.Input(shape=(1,), name="sex", dtype="int64")
sex = keras.Input(shape=(1,), name="sex", dtype="int64")
sex = keras.Input(shape=(1,), name="sex", dtype="int64")
sex = keras.Input(shape=(1,), name="sex", dtype="int64")
sex = keras.Input(shape=(1,), name="sex", dtype="int64")

sex	age	sibsp	parch	fare	embarked	boat	home.dest

TypeError: ('Keyword argument not understood:', 'output_mode')

In [13]:
all_features = layers.concatenate(
    [
        sex_encoded,
        cp_encoded,
        fbs_encoded,
        restecg_encoded,
        exang_encoded,
        slope_encoded,
        ca_encoded,
        thal_encoded,
        age_encoded,
        trestbps_encoded,
        chol_encoded,
        thalach_encoded,
        oldpeak_encoded,
    ]
)


NameError: name 'sex_encoded' is not defined

In [72]:
all_inputs = [
    keras.Input(shape=(1,), name='sex')
]
x = layers.Dense(32, activation="relu")(all_inputs)
x = layers.Dropout(0.5)(x)
output = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(all_inputs, output)
model.compile("adam", "binary_crossentropy", metrics=["accuracy"])

TypeError: Dimension value must be integer or None or have an __index__ method, got value 'TensorShape([None, 1])' with type '<class 'tensorflow.python.framework.tensor_shape.TensorShape'>'

In [15]:
# `rankdir='LR'` is to make the graph horizontal.
keras.utils.plot_model(model, show_shapes=True, rankdir="LR")

NameError: name 'model' is not defined

In [16]:
model.fit(train_ds, epochs=50, validation_data=val_ds)


NameError: name 'model' is not defined

In [17]:
sample = {
    "age": 60,
    "sex": 1,
    "cp": 1,
    "trestbps": 145,
    "chol": 233,
    "fbs": 1,
    "restecg": 2,
    "thalach": 150,
    "exang": 0,
    "oldpeak": 2.3,
    "slope": 3,
    "ca": 0,
    "thal": "fixed",
}

input_dict = {name: tf.convert_to_tensor([value]) for name, value in sample.items()}
predictions = model.predict(input_dict)

print(
    "This particular patient had a %.1f percent probability "
    "of having a heart disease, as evaluated by our model." % (100 * predictions[0][0],)
)

NameError: name 'model' is not defined

In [29]:
train_ds

<ShuffleDataset shapes: ({pclass: (), sex: (), age: (), sibsp: (), parch: (), fare: (), embarked: (), boat: (), home.dest: ()}, ()), types: ({pclass: tf.float64, sex: tf.string, age: tf.float64, sibsp: tf.float64, parch: tf.float64, fare: tf.float64, embarked: tf.string, boat: tf.string, home.dest: tf.string}, tf.string)>

In [84]:
from tensorflow.keras.models import Sequential
model = Sequential()

In [85]:
model.add(layers.Dense(8, activation='relu'))

model.add(layers.Dense(8, activation='relu'))

model.add(layers.Dense(1, activation='sigmoid'))

In [92]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
                   
model.fit(X_train, y_train,epochs=50, batch_size=32, verbose=1)

Epoch 1/50
17/17 [==============================] - 1s 1ms/step - loss: 0.4412 - accuracy: 0.9269
Epoch 2/50
17/17 [==============================] - 0s 1ms/step - loss: 0.3812 - accuracy: 0.9806
Epoch 3/50
17/17 [==============================] - 0s 1ms/step - loss: 0.4154 - accuracy: 0.9772
Epoch 4/50
17/17 [==============================] - 0s 1ms/step - loss: 0.4096 - accuracy: 0.9723
Epoch 5/50
17/17 [==============================] - 0s 1ms/step - loss: 0.3363 - accuracy: 0.9846
Epoch 6/50
17/17 [==============================] - 0s 1ms/step - loss: 0.3701 - accuracy: 0.9696
Epoch 7/50
17/17 [==============================] - 0s 1ms/step - loss: 0.4221 - accuracy: 0.9709
Epoch 8/50
17/17 [==============================] - 0s 1ms/step - loss: 0.3616 - accuracy: 0.9807
Epoch 9/50
17/17 [==============================] - 0s 1ms/step - loss: 0.3356 - accuracy: 0.9750
Epoch 10/50
17/17 [==============================] - 0s 1ms/step - loss: 0.3142 - accuracy: 0.9841
Epoch 11/50
17/17 [

In [89]:
y_train = np.asarray(y_train).astype('int64')

In [90]:
y_train

array([0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1,
       1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1,
       1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0,
       1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1,
       1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1,